In [ ]:
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import Dataset, DataLoader
from torchvision import transforms

from torchvision.models import resnet50
from torchvision.models import ResNet50_Weights

from typing import cast
from tqdm import tqdm
from math import inf
import numpy as np
import pandas as pd
import os

from PIL import Image
from sklearn.model_selection import train_test_split

In [1]:
data_path = os.path.join(os.getcwd(), "data")
model_path = os.path.join(os.getcwd(), "models")
device = "cuda" if torch.cuda.is_available() else "cpu"

df = pd.read_csv(rf"{data_path}/trash.csv")
df = df.reset_index(drop=True)
print(df.head())

   Unnamed: 0                                          file_path  is_organic
0           1  /home/espacio/ml/sem5-proj/data/d1/DATASET/TRA...        True
1          25  /home/espacio/ml/sem5-proj/data/d1/DATASET/TRA...        True
2          26  /home/espacio/ml/sem5-proj/data/d1/DATASET/TRA...        True
3          27  /home/espacio/ml/sem5-proj/data/d1/DATASET/TRA...        True
4          31  /home/espacio/ml/sem5-proj/data/d1/DATASET/TRA...        True


### Split the dataset

In [2]:
df_train, df_test = train_test_split(df, test_size=0.1, stratify=df["is_organic"], random_state=42)
df_train, df_test=cast(pd.DataFrame,df_train), cast(pd.DataFrame, df_test)
df_train = df_train.reset_index(drop=True)
df_test = df_test.reset_index(drop=True)
print(len(df_train))
print(len(df_test))

249093
27677


### Define the data loade

In [ ]:
class TrashDataset(Dataset):
    def __init__(self, df: pd.DataFrame, transform = None) -> None:
        self.df=df
        self.transform=transform

    def __len__(self):
        return len(self.df)

    def __getitem__(self, idx):
        file_path = self.df.loc[idx, "file_path"]
        label = int(self.df.loc[idx, "is_organic"])
        img = Image.open(file_path).convert("RGB")

        if self.transform:
            img=self.transform(img)
        else:
            img=transforms.ToTensor()(img)  # scale 0-1
        return img, label

trash_transforms = transforms.Compose([
    transforms.Resize((128, 128)),
    transforms.ToTensor()
])

train_dataset = TrashDataset(df_train,trash_transforms)
test_dataset = TrashDataset(df_test,trash_transforms)

train_loader = DataLoader(train_dataset, batch_size=64)
test_loader = DataLoader(test_dataset, batch_size=64)

### Define the model

In [ ]:
model = resnet50(weights=ResNet50_Weights.DEFAULT)
for param in model.parameters():
    param.requires_grad=False

num_ftrs = model.fc.in_features
model.fc=nn.Linear(num_ftrs,1)
model=model.to(device)

### Define the optimize

In [ ]:
criterion = nn.BCEWithLogitsLoss()
optimizer = optim.Adam(model.fc.parameters(), lr=4e-3)

### Train the model

In [3]:
n_epochs = 200
best_loss = inf

for epoch in range(n_epochs):
    train_loss = 0.0
    test_loss = 0.0
    total = 0
    test_correct = 0

    model.train()
    for (images,labels) in tqdm(train_loader):
        images, labels = images.to(device), labels.to(device)

        optimizer.zero_grad()
        outputs = model(images)

        loss = criterion(outputs,labels.float().unsqueeze(1))
        loss.backward()
        optimizer.step()

        train_loss += loss.item()

    train_loss /= len(train_loader)
    print(f"Epoch {epoch+1} (Train):\n")
    print(f"\tTrain Loss: {train_loss:.4f}")

    model.eval()
    with torch.no_grad():
        for (images,labels) in tqdm(test_loader):
            images, labels=images.to(device), labels.to(device)

            outputs = model(images)
            loss = criterion(outputs,labels.float().unsqueeze(1))
            predictions = (outputs>0).squeeze().long()

            test_loss += loss.item()
            total += labels.size(0)
            test_correct += (predictions==labels).sum()

    test_loss /= len(test_loader)
    print(f"Epoch {epoch+1} (Test):\n")
    print(f"\tTest Loss: {test_loss:.4f}")
    print(f"\tTest Accuracy: {100*test_correct/total:.2f}%")

    if (test_loss < best_loss):
        best_loss = test_loss
        best_model_path = rf"{model_path}/feature_extraction_model.pth"
        torch.save(model.state_dict(), best_model_path)

---------------------------------------------------------------------------
KeyboardInterrupt                         Traceback (most recent call last)
Cell In[78], line 11
      8 test_correct = 0
     10 model.train()
---> 11 for (images,labels) in tqdm(train_loader):
     12     images, labels = images.to(device), labels.to(device)
     14     optimizer.zero_grad()

File ~/.conda/envs/ml/lib/python3.13/site-packages/tqdm/std.py:1181, in tqdm.__iter__(self)
   1178 time = self._time
   1180 try:
-> 1181     for obj in iterable:
   1182         yield obj
   1183         # Update and possibly print the progressbar.
   1184         # Note: does not call self.update(1) for speed optimisation.

File ~/.conda/envs/ml/lib/python3.13/site-packages/torch/utils/data/dataloader.py:734, in _BaseDataLoaderIter.__next__(self)
    731 if self._sampler_iter is None:
    732     # TODO(https://github.com/pytorch/pytorch/issues/76750)
    733     self._reset()  # type: ignore[call-arg]
--> 734 data = 

### Load best model

In [4]:
model = resnet50(weights=ResNet50_Weights.DEFAULT)
num_ftrs = model.fc.in_features

model.fc = nn.Linear(num_ftrs, 1)
model.load_state_dict(torch.load(rf"{model_path}/feature_extraction_model.pth", map_location=device))

model.fc = nn.Identity() # type: ignore
model.eval()
model.to(device)

ResNet(
  (conv1): Conv2d(3, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False)
  (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (relu): ReLU(inplace=True)
  (maxpool): MaxPool2d(kernel_size=3, stride=2, padding=1, dilation=1, ceil_mode=False)
  (layer1): Sequential(
    (0): Bottleneck(
      (conv1): Conv2d(64, 64, kernel_size=(1, 1), stride=(1, 1), bias=False)
      (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (conv2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn2): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (conv3): Conv2d(64, 256, kernel_size=(1, 1), stride=(1, 1), bias=False)
      (bn3): BatchNorm2d(256, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (relu): ReLU(inplace=True)
      (downsample): Sequential(
        (0): Conv2d(64, 256, kernel_size=(1, 1), stride=(1, 

### Generate feature vectors

In [5]:
full_dataset = TrashDataset(df, trash_transforms)
full_loader = DataLoader(full_dataset, batch_size=64, shuffle=False)

all_features = []
all_labels = []

with torch.no_grad():
    for (images, labels) in tqdm(full_loader, desc="Extracting features"):
        images = images.to(device)

        features = model(images)

        all_features.append(features.cpu().numpy())
        all_labels.append(labels.numpy())

all_features = np.concatenate(all_features, axis=0)
all_labels = np.concatenate(all_labels, axis=0)

print("Final feature shape:", all_features.shape)
print("Final labels shape:", all_labels.shape)

features_df = pd.DataFrame(all_features)
features_df["label"] = all_labels

output_path = os.path.join(data_path, "final.csv")
features_df.to_csv(output_path, index=False)

print(f"Saved feature vectors + labels to {output_path}")

Final feature shape: (276770, 2048)
Final labels shape: (276770,)
Saved feature vectors + labels to /home/espacio/ml/sem5-proj/data/final.csv
